<a href="https://colab.research.google.com/github/knickka/Comment-Toxicity/blob/main/Comment_toxicity_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np


In [3]:
path = '/content/drive/MyDrive/Colab Notebooks/Toxic_Comment_Detection/toxci_comment_train.csv'
df = pd.read_csv(path)


In [ ]:
df.iloc[43]['comment_text']

In [ ]:
df[df.columns[2:]].iloc[43]

## Preprocess

In [6]:
#imports
from tensorflow.keras.layers import TextVectorization

In [7]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [8]:
MAX_WORDS = 200000 #number of words in our vocab


In [9]:
vectorizer = TextVectorization(max_tokens=MAX_WORDS,
                                  output_sequence_length = 1800,
                                  output_mode = 'int')

In [10]:
vectorizer.adapt(X.values)

In [11]:
vectorized_text = vectorizer(X.values)

In [ ]:
len(X)

In [ ]:
vectorized_text

In [77]:
def Save_Vectorizer(vectorizer,path):
  vectorizer_model = tf.keras.models.Sequential()
  vectorizer_model.add(tf.keras.Input(shape=(1,),dtype=tf.string))
  vectorizer_model.add(vectorizer)
  res = vectorizer_model.summary()

  vectorizer_model.save(path,save_format='tf')
  print(res)

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/Toxic_Comment_Detection/vectorizer'
Save_Vectorizer(vectorizer,path)

In [14]:
#MCSHBP - map, cache, shuffle, batch prefetch from_tensor_slices, from_list

dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [ ]:
dataset.as_numpy_iterator().next()

In [16]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [17]:
train_generator = train.as_numpy_iterator()

In [21]:
#imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional, Embedding

In [22]:
model = Sequential()
model.add(Embedding(MAX_WORDS+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='softmax'))


In [23]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam')

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                      

In [30]:
history = model.fit(train,epochs=1,validation_data=val)

6981/6981 [==============================] - 15331s 2s/step - loss: 0.0617 - val_loss: 0.0466


In [60]:
batch_x,batch_y = test.as_numpy_iterator().next()


In [ ]:
batch_y

In [ ]:
(model.predict(batch_x)>0.5).astype(int)

In [64]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [65]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
from tensorflow.python.ops.array_ops import batch_gather
for batch in test.as_numpy_iterator():
  x_true,y_true = batch

  yhat = model.predict(x_true)

  y_true = y_true.flatten()
  yhat = yhat.flatten()

  pre.update_state(y_true,yhat)
  re.update_state(y_true,yhat)
  acc.update_state(y_true,yhat)

In [ ]:
print(re.result().numpy())
print(pre.result().numpy())
print(acc.result().numpy())

In [ ]:
path_m = '/content/drive/MyDrive/Colab Notebooks/Toxic_Comment_Detection/model'
model.save(path_m,save_format='tf')

In [ ]:
features = df.columns[2:].values.tolist()
features

In [83]:
def return_vec(path):
  vec_model = tf.keras.models.load_model(path)
  vectorizer = vec_model.layers[0]
  return vectorizer

In [84]:
vectorizer = return_vec(path)

In [86]:
model = tf.keras.models.load_model(path_m)